<a href="https://colab.research.google.com/github/kurogo-nguyen/ern/blob/main/findError.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import

In [ ]:
! pip install mne
# ! pip install AutoReject
! pip install h5io

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.5 MB 5.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import mne
import matplotlib
import pandas as pd
from os import listdir, path

# parameter

In [ ]:
DATA_PATH = '/content/drive/MyDrive/DataHMI'

HOLD_TIME = 1.5

In [ ]:
def getError(id: int, sample: str, path=DATA_PATH):
    df = pd.read_csv(f'{path}/{id}/sample{sample}/ET.csv', keep_default_na=False)

    startTime = df.iloc[0].TimeStamp

    df.drop(['x','y'], axis=1, inplace=True)

    # # Remove non typing segment
    df = df[df.sentence!= 'MainMenu']

    df['prev_sentence'] = df.sentence.shift(1)
    df = df[df.sentence != df.prev_sentence]
    df = df[~df['character typing'].isin(['', 'START_ET_STREAM'])]

    df['onset'] = df['TimeStamp'] - startTime
    # df.rename(columns={'character typing':'event',}, inplace=True)
    # df['isCorrect'] = (df['character typing'] != '⌫')

    df = df[df['character typing'].isin(['⌫','⌧']) ]

    return df

In [ ]:
def getOnset(id: int, sample: str, path=DATA_PATH):
    df = pd.read_csv(f'{path}/{id}/sample{sample}/ET.csv', keep_default_na=False)

    startTime = df.iloc[0].TimeStamp

    df.drop(['x','y'], axis=1, inplace=True)

    # Remove non typing segment
    df = df[df.sentence!= 'MainMenu']

    df['prev_sentence'] = df.sentence.shift(1)
    df = df[df.sentence != df.prev_sentence]
    df = df[~df['character typing'].isin(['', 'START_ET_STREAM'])]

    df['onset'] = df['TimeStamp'] - startTime
    # df.rename(columns={'character typing':'event',}, inplace=True)
    df['isCorrect'] = (df['character typing'] != '⌫')

    return df

In [ ]:
def insertERN(id: int, idx: int, sample: str, prev_onset: float, del_onset: float, 
              sentence_error: str, next_sentence: str, df: pd.DataFrame)-> pd.DataFrame:
    onset = prev_onset+HOLD_TIME
    duration = del_onset - prev_onset-HOLD_TIME
    row = {'id':id, 'sample': sample, 'onset': onset, 
           'duration': duration, 
           'sentence error': sentence_error, 
           'next sentence': next_sentence, 'idx':idx}

    return df.append(row,ignore_index=True)

In [ ]:
def smt(id: int, sample: str, idx: int) -> pd.DataFrame:

    df = pd.read_csv(f'{DATA_PATH}/{id}/sample{sample}/ET.csv', keep_default_na=False)

    startTime = df.iloc[0].TimeStamp

    df = df[df.sentence!= 'MainMenu']
    df['prev_sentence'] = df.sentence.shift(1)
    df = df[df.sentence != df.prev_sentence]

    timestamps = df['TimeStamp'].to_list() - startTime - HOLD_TIME
    charTypings = df['character typing'].to_list()
    sentences = df['sentence'].to_list()

    # typing_onsets = []
    # typing_durations = []
    # typing_descriptions = []
    m_ERN_df = pd.DataFrame(columns=['id', 'sample', 'onset', 'duration', 'sentence error', 'next sentence', 'idx'])

    for i in range(2,len(timestamps)):
        if charTypings[i] in ['⌫', '⌧']:
            m_ERN_df = insertERN(id, idx, sample, timestamps[i-1], timestamps[i], sentences[i-1], sentences[i], m_ERN_df)
            
    #         typing_onsets.append(timestamps[i-1] + hold_time)
    #         typing_durations.append(timestamps[i]-timestamps[i-1]-hold_time)
    #         typing_descriptions.append('has ERN')

    #     typing_onsets.append(timestamps[i])
    #     typing_durations.append(hold_time)
    #     typing_descriptions.append(charTypings[i])

    # raw = mne.io.read_raw(f'{DATA_PATH}/K{id}/sample{sample}/EEG.edf', preload=True, verbose=0)

    # typing_anotations = mne.Annotations(onset=typing_onsets, 
    #                                 duration=typing_durations, 
    #                                 description=typing_descriptions, 
    #                                 orig_time=raw.info['meas_date'])

    
    # raw.set_annotations(typing_anotations)
    # l_raw.append(raw)
    # for i in range(len(m_ERN_df.idx)):
    #     figs.append(raw.plot(start=m_ERN_df.onset[i]-0.5, duration=m_ERN_df.duration[i]+1, scalings=0.05, order=frontal_channels_order, lowpass=10, highpass=0.1))

    return m_ERN_df

In [ ]:
ERN_df = pd.DataFrame(columns=['id', 'sample', 'onset', 'duration', 'sentence error', 'next sentence', 'idx'])
ERN_df

,id,sample,onset,duration,sentence error,next sentence,idx


In [ ]:
# l_df = []
# l_raw = []
# count=0
# p = []
idx=0
for id in listdir(DATA_PATH):
    for sample in range(1,10):
        if path.exists(f'{DATA_PATH}/K333/sample{sample}'):
            ERN_df=ERN_df.append(smt('K333', sample, idx))
            idx=idx+1
            # df = getError(id, sample, DATA_PATH)
            # if '⌫' in df['character typing'].unique():
            #     # print(f'{count} {id} {sample} done')
            #     ERN_df.m
            #     p.append([id,sample])
            #     l_df.append(df)
            #     l_raw.append(loadRaw(id, sample, ALS_DATA_PATH))
            #     count=count+1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
ERN_df

,id,sample,onset,duration,sentence error,next sentence,idx
0,K333,4,103.035050,3.662215,nâng chân t,nâng chân,3
0,K333,9,80.943562,0.895394,tôi muốn đi t,tôi muốn đi,8
0,K333,4,103.035050,3.662215,nâng chân t,nâng chân,12
0,K333,9,80.943562,0.895394,tôi muốn đi t,tôi muốn đi,17
0,K333,4,103.035050,3.662215,nâng chân t,nâng chân,21
...,...,...,...,...,...,...,...
0,K333,9,80.943562,0.895394,tôi muốn đi t,tôi muốn đi,629
0,K333,4,103.035050,3.662215,nâng chân t,nâng chân,633
0,K333,9,80.943562,0.895394,tôi muốn đi t,tôi muốn đi,638
0,K333,4,103.035050,3.662215,nâng chân t,nâng chân,642


In [ ]:
ERN_df

,id,sample,onset,duration,sentence error,next sentence,idx
0,K333,4,103.035050,3.662215,nâng chân t,nâng chân,39
1,K333,9,80.943562,0.895394,tôi muốn đi t,tôi muốn đi,44
2,K355,9,110.207229,0.759395,tôi muốn đi vệ să,tôi muốn đi vệ s,53
3,K334,1,126.545099,12.707590,nâng tay p,nâng tay,63
4,K344,3,105.818152,2.511677,nâng n,nâng,83
...,...,...,...,...,...,...,...
0,K323,2,113.020148,2.711532,na,n,523
0,K323,8,85.673918,2.012782,tôi muốn n,tôi muốn,529
1,K323,8,116.591668,1.559611,tôi muốn uống nu,tôi muốn uống n,529
0,K323,9,69.926763,4.542755,tôi muống,tôi muốn,530


In [ ]:
ERN_df.reset_index(drop=True, inplace=True)

In [ ]:
ERN_df

,id,sample,onset,duration,sentence error,next sentence,idx
0,K333,4,103.035050,3.662215,nâng chân t,nâng chân,39
1,K333,9,80.943562,0.895394,tôi muốn đi t,tôi muốn đi,44
2,K355,9,110.207229,0.759395,tôi muốn đi vệ să,tôi muốn đi vệ s,53
3,K334,1,126.545099,12.707590,nâng tay p,nâng tay,63
4,K344,3,105.818152,2.511677,nâng n,nâng,83
...,...,...,...,...,...,...,...
120,K323,2,113.020148,2.711532,na,n,523
121,K323,8,85.673918,2.012782,tôi muốn n,tôi muốn,529
122,K323,8,116.591668,1.559611,tôi muốn uống nu,tôi muốn uống n,529
123,K323,9,69.926763,4.542755,tôi muống,tôi muốn,530


In [ ]:
ERN_df

,id,sample,onset,duration,sentence error,next sentence,idx
0,K333,4,103.035050,3.662215,nâng chân t,nâng chân,39
1,K333,9,80.943562,0.895394,tôi muốn đi t,tôi muốn đi,44
2,K355,9,110.207229,0.759395,tôi muốn đi vệ să,tôi muốn đi vệ s,53
3,K334,1,126.545099,12.707590,nâng tay p,nâng tay,63
4,K344,3,105.818152,2.511677,nâng n,nâng,83
5,K358,8,68.297819,0.695583,tôi l,tôi,178
6,K358,9,66.564346,0.895739,tôi l,tôi,179
7,K358,9,103.588414,1.648203,tôi muốn đi ma,tôi muốn đi m,179
8,K372,1,75.067599,5.938171,d,,180
9,K300,2,133.273021,1.715532,nâng tay phải,,190


In [ ]:
ERN_df[['id','sample']].drop(33).drop_duplicates().reset_index(drop=True).to_csv('/content/drive/MyDrive/DataHMI/error3.csv')